### Step 1. 라이브러리 버전

In [1]:
import glob, os
import tensorflow

print(tensorflow.__version__)

2.6.0


### Step 2. 데이터 읽어오기

In [2]:
txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'

In [3]:
txt_list = glob.glob(txt_file_path)

In [4]:
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, 'r') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print('데이터 크기 :', len(raw_corpus))
print('Examples : \n', raw_corpus[:3])

데이터 크기 : 187088
Examples : 
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### Step 3. 데이터 정제

In [5]:
import re

In [6]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [7]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    
    if len(preprocessed_sentence.split(' ')) <= 15:
        corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [8]:
corpus.__len__()

156013

In [9]:
import tensorflow as tf

In [12]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f9324420460>


In [13]:
# tokenizer.index_word: 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 (Ex. {index: '~~', index: '~~', ...})
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


### Step 4. 평가 데이터셋 분리

In [15]:
from sklearn.model_selection import train_test_split

In [14]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


In [16]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size= 0.2)

In [20]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### Step 5. 모델 학습

In [21]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [22]:
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 2.76809616e-04, -1.64152094e-04,  2.67026859e-04, ...,
         -5.51319281e-05, -6.67849745e-05, -1.18391408e-05],
        [ 5.49796736e-04, -3.30991577e-04,  5.42493479e-04, ...,
         -1.67273829e-04, -1.12152120e-04,  1.48283987e-04],
        [ 6.75649673e-04, -5.50219498e-04,  6.53348921e-04, ...,
         -2.75901315e-04, -3.38702637e-04, -1.81055701e-04],
        ...,
        [ 1.20358495e-03, -3.26063426e-04,  1.11383828e-03, ...,
         -7.51923071e-05, -8.10368045e-04, -8.78806168e-04],
        [ 1.53952010e-03, -4.57625254e-04,  1.09998696e-03, ...,
         -3.67376022e-04, -1.08685333e-03, -1.07017322e-03],
        [ 1.83737359e-03, -5.74726786e-04,  1.08697009e-03, ...,
         -6.88949600e-04, -1.32381474e-03, -1.25818013e-03]],

       [[ 2.76809616e-04, -1.64152094e-04,  2.67026859e-04, ...,
         -5.51319281e-05, -6.67849745e-05, -1.18391408e-05],
        [ 3.33809468e-04, -4.63741628e-04,  5

In [23]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [24]:
optimizer = tf.keras.optimizers.Adam() 
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none')
model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
609/609 [==============================] - 112s 180ms/step - loss: 3.4006
Epoch 2/30
609/609 [==============================] - 111s 182ms/step - loss: 2.9590
Epoch 3/30
609/609 [==============================] - 111s 182ms/step - loss: 2.7853
Epoch 4/30
609/609 [==============================] - 111s 182ms/step - loss: 2.6540
Epoch 5/30
609/609 [==============================] - 111s 182ms/step - loss: 2.5414
Epoch 6/30
609/609 [==============================] - 111s 182ms/step - loss: 2.4403
Epoch 7/30
609/609 [==============================] - 111s 182ms/step - loss: 2.3467
Epoch 8/30
609/609 [==============================] - 111s 182ms/step - loss: 2.2599
Epoch 9/30
609/609 [==============================] - 111s 182ms/step - loss: 2.1785
Epoch 10/30
609/609 [==============================] - 111s 182ms/step - loss: 2.1014
Epoch 11/30
609/609 [==============================] - 111s 182ms/step - loss: 2.0291
Epoch 12/30
609/609 [==============================] - 111s 182

### Step 6. 모델 평가

In [26]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): 
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [28]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you mom , you always be my favorite girl . <end> '

In [29]:
generate_text(model, tokenizer, init_sentence="<start> i will", max_len=20)

'<start> i will survive , i will survive <end> '

In [30]:
generate_text(model, tokenizer, init_sentence="<start> You are", max_len=20)

'<start> you are the only thing that keeps me goin <end> '

In [31]:
generate_text(model, tokenizer, init_sentence="<start> It is", max_len=20)

'<start> it is not too late <end> '

In [34]:
generate_text(model, tokenizer, init_sentence="<start> You can", max_len=20)

'<start> you can t be wonderin why they always stayin dead broke <end> '

### 회고

#### 1. NLP에 익숙치 않았는데 전반적인 과정을 이해하고, 배울 수 있어서 좋았습니다.
#### 2. 데이터 전처리 과정에 여전히 의문점이 남아있고, 이를 조금 더 조사해볼 계획이다. ( 임베딩이 어떻게 일어나는지에 대해 )
#### 3. 출력 시 작은 따옴표나 마침표가 잘 찍히지 않는 문제점을 발견하였고, 이를 개선해 볼 계획이다.